In [1]:
# Import the pandas library for reading and manipulating your data
# Anywhere you see "pd" in this notebook it's a reference to the pandas library
import pandas as pd
# Extra step to ensure that pandas plays nice with matplotlib
pd.plotting.register_matplotlib_converters()
# Import the numpy library for running calculations on your data
# Anywhere you see "np" in this notebook it's a reference to the numpy library
import numpy as np
# Import some components of the matplotlib library for plotting your data
# Anywhere you see "plt" or "mpimg" in this notebook it's a reference to the "pyplot" and "image" packages from matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Import seaborn library for making your plots pretty!
# Anywhere you see "sns" in this notebook it's a reference to the seaborn library
import seaborn as sns
# Set some default plotting parameters using seaborn
sns.set()

In [2]:
# Store file name as string
csv_string_2016 = "Parking_Violations_Issued_-_Fiscal_Year_2016.csv"
csv_string_2017 = "Parking_Violations_Issued_-_Fiscal_Year_2017.csv"

# Convert full dataset csvs to dataframes
df_16 = pd.read_csv(csv_string_2016)
df_17 = pd.read_csv(csv_string_2017)



/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,34,36,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
# Concatenate the 2016 and 2017 data
df = pd.concat([df_16, df_17], ignore_index=True)

In [31]:
# Identify columns from the dataset that we are interested in keeping
cols_to_keep = ['Plate ID',
                'Registration State',
                'Plate Type',
                'Issue Date',
                'Violation Code',
                'Vehicle Body Type',
                'Vehicle Make',
                'Vehicle Color',
                'Violation Time',
                'Violation County',
                'Vehicle Year']
# Remove the unnecessary columns
cols_to_remove = [col for col in df.columns if col not in cols_to_keep]
df = df.drop(columns = cols_to_remove)

# Rename the columns to have underscores instead of spaces for easier access
cols_to_keep_rename = {}
for col in cols_to_keep:
    renamed = col.replace(" ", "_").lower()
    cols_to_keep_rename[col] = renamed
df = df.rename(columns = cols_to_keep_rename)

In [32]:
# Filter data by matching values in each column

# Store the codes for color, county, body, make, and violation to ease data cleaning
color_codes = ['BK', 'BL', 'BR', 'GL', 'GY',
               'MR', 'OR', 'PK', 'PR', 'RD',
               'TN', 'WH', 'YW','BLACK', 'BLUE', 
               'GREY', 'GREEN', 'RED', 
               'WHITE', 'YELLOW', 'SILVE', 'BROWN', 'GOLD', 'PINK']

county_codes = ["BX", "K", "NY", "Q", "R"]

body_codes_to_keep = ['CONV', 'SUBN', '4DSD',
              '2DSD', 'PICK', 'VAN']

vehicle_makes_to_keep  = ['ACURA', 'AUDI', 'BMW', 'BUICK', 'CADIL',
                          'CHEVR', 'CHRYS', 'DODGE', 'FIAT', 'FORD',
                          'GMC', 'HONDA', 'HUMME', 'HYUND', 'INFIN',
                          'ISUZU', 'JAGUA', 'JEEP', 'KIA', 'LEXUS',
                          'LINCO', 'MASSA', 'MAZDA', 'ME/BE', 'MERCU',
                          'MINI', 'MITSU', 'NISSA', 'OLDSM', 'PLYMO',
                          'PONTI', 'PORSC', 'ROVER', 'SAAB', 'SATUR',
                          'SMART', 'SUBAR', 'SUZUK', 'TESLA', 'TOYOT',
                          'VOLKS', 'VOLVO']

violation_codes_to_keep = [20, 21, 37, 38, 40]

# Apply filters
print(f'{len(df)} rows before filtering')
# Clean data by filtering out rows that do not conform to lists above
df = df[df.vehicle_color.isin(color_codes)]
print(f'{len(df)} rows after color filtering')
df = df[df.vehicle_body_type.isin(body_codes_to_keep)]
print(f'{len(df)} rows after body type filtering')
df = df[df.violation_code.isin(violation_codes_to_keep)]
print(f'{len(df)} rows after violation code filtering')
df = df[df.vehicle_make.isin(vehicle_makes_to_keep)]
print(f'{len(df)} rows after vehicle make filtering')
df = df[df.violation_county.isin(county_codes)]
print(f'{len(df)} rows after county code filtering')
df = df[df.plate_type == 'PAS']
print(f'{len(df)} rows after plate type filtering')
df = df[df.vehicle_year != 0]
df = df[df.vehicle_year <= 2017]
print(f'{len(df)} rows after vehicle year filtering')
df = df[df.issue_date.astype(str).str.contains("2016")]
print(f'{len(df)} rows after restricting to 2016')


21429927 rows before filtering
19393280 rows after color filtering
16262245 rows after body type filtering
7399603 rows after violation code filtering
7238002 rows after vehicle make filtering
7231458 rows after county code filtering
6026768 rows after plate type filtering
4327330 rows after vehicle year filtering
2043444 rows after restricting to 2016


In [33]:
# Convert time to proper format for to_datetime
df['violation_time'] = df['violation_time'].apply(\
             lambda x: "12"+x[2:]+"M" if x[:2] == "00" else x+"M")
# Combine columns into new timestamp column
df['timestamp'] = (df.issue_date.astype(str) + \
                            " " + df.violation_time.astype(str))

# Convert timestamp col into datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], \
                             errors = 'coerce', format = "%m/%d/%Y %I%M%p")

# Drop NaT rows that were unable to convert to datetime
df.dropna(inplace=True)
# Drop any duplicate rows
df.drop_duplicates(keep=False,inplace=True)
# Sort by timestamp
df.sort_values(by=['timestamp'], inplace=True)
# Set timestamp as index to get rid of goofy index values
df = df.set_index('timestamp')


In [34]:
# Create color map for values to change and values to keep the same (annoyingly have to map all)
color_map = {'BK':'BLACK', 'BL':'BLUE', 'BR':'BROWN', 'GL':'GOLD', 'GY':'GRAY',
            'MR':'MAROON', 'OR':'ORANGE', 'PK':'PINK', 'PR':'PURPLE', 'RD':'RED',
            'TN':'TAN', 'WH':'WHITE', 'YW':'YELLOW',
            'GREY':'GRAY', 'SILVE':'SILVER', 'BLACK':'BLACK', 'BLUE':'BLUE', 'BROWN':'BROWN', 
            'GOLD':'GOLD', 'GRAY':'GRAY',
            'MAROON':'MAROON', 'ORANGE':'ORANGE', 'PINK':'PINK', 'PURPLE':'PURPLE', 'RED':'RED',
            'TAN':'TAN', 'WHITE':'WHITE', 'YELLOW':'YELLOW',
            'GRAY':'GRAY', 'SILVER':'SILVER'}
df.vehicle_color = df.vehicle_color.map(color_map)
# Map county and violation code as well
county_map = {'K':'Brooklyn', 'Q':'Queens', 'NY':'Manhattan', 'BX':'Bronx', 'R':'Staten Island'}
df.violation_county = df.violation_county.map(county_map)
violation_codes_map = {20:'NO PARKING ZONE', 21:'STREET CLEANING', 37:'NO METER RECEIPT', 
                       38:'EXPIRED METER', 40:'FIRE HYDRANT'}
df.violation_code = df.violation_code.map(violation_codes_map)

In [35]:
# Verify that things appear to have worked as intended, i.e. sorting, mapping and timestamp conversion
df.head(100)

,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,violation_time,violation_county,vehicle_color,vehicle_year
timestamp,,,,,,,,,,,
2016-01-01 00:40:00,GXA7174,NY,PAS,01/01/2016,FIRE HYDRANT,SUBN,INFIN,1240AM,Bronx,BLUE,2009.0
2016-01-01 00:50:00,GYK1668,NY,PAS,01/01/2016,NO PARKING ZONE,SUBN,FORD,1250AM,Bronx,GRAY,2003.0
2016-01-01 01:34:00,HDU1328,NY,PAS,01/01/2016,FIRE HYDRANT,SUBN,NISSA,0134AM,Manhattan,BLACK,2011.0
2016-01-01 01:53:00,GPU1150,NY,PAS,01/01/2016,FIRE HYDRANT,SUBN,NISSA,0153AM,Brooklyn,BLACK,1999.0
2016-01-01 02:41:00,FYK7567,NY,PAS,01/01/2016,FIRE HYDRANT,SUBN,BMW,0241AM,Bronx,WHITE,2014.0
2016-01-01 03:18:00,651103N,NJ,PAS,01/01/2016,FIRE HYDRANT,SUBN,INFIN,0318AM,Bronx,BLACK,2009.0
2016-01-01 03:20:00,GVY9503,NY,PAS,01/01/2016,NO PARKING ZONE,SUBN,CHEVR,0320AM,Bronx,WHITE,2004.0
2016-01-01 03:50:00,HBZ7615,NY,PAS,01/01/2016,FIRE HYDRANT,SUBN,CHRYS,0350AM,Bronx,GRAY,2005.0
2016-01-01 05:00:00,CLW3803,NY,PAS,01/01/2016,FIRE HYDRANT,SUBN,NISSA,0500AM,Bronx,BLUE,1998.0


In [36]:
df.tail(100)

,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,violation_time,violation_county,vehicle_color,vehicle_year
timestamp,,,,,,,,,,,
2016-12-31 17:17:00,HHJ9313,NY,PAS,12/31/2016,NO METER RECEIPT,4DSD,LINCO,0517PM,Manhattan,RED,2009.0
2016-12-31 17:18:00,HEA6273,NY,PAS,12/31/2016,EXPIRED METER,SUBN,FORD,0518PM,Queens,BLUE,2002.0
2016-12-31 17:18:00,GYR9789,NY,PAS,12/31/2016,NO METER RECEIPT,SUBN,TOYOT,0518PM,Manhattan,BLUE,2015.0
2016-12-31 17:19:00,GCR1290,NY,PAS,12/31/2016,FIRE HYDRANT,SUBN,TOYOT,0519PM,Brooklyn,SILVER,2006.0
2016-12-31 17:19:00,GVG7521,NY,PAS,12/31/2016,EXPIRED METER,SUBN,SUBAR,0519PM,Queens,RED,2015.0
2016-12-31 17:19:00,HDW8645,NY,PAS,12/31/2016,EXPIRED METER,SUBN,CHRYS,0519PM,Manhattan,GRAY,2014.0
2016-12-31 17:20:00,EUT3300,NY,PAS,12/31/2016,NO METER RECEIPT,4DSD,VOLKS,0520PM,Manhattan,GRAY,2009.0
2016-12-31 17:20:00,HCB5152,NY,PAS,12/31/2016,EXPIRED METER,SUBN,NISSA,0520PM,Queens,BLACK,2012.0
2016-12-31 17:20:00,HKM5570,NY,PAS,12/31/2016,EXPIRED METER,SUBN,TOYOT,0520PM,Queens,BLUE,2010.0


In [37]:
# If timestamp conversion appears successful, drop the unneeded violation_time and issue_date columns
# (also drop plate type as no longer needed)
df.drop(columns = ['violation_time', 'issue_date', 'plate_type'], inplace=True)
# Rename county to borough
df = df.rename(columns = {'violation_county':'borough'})
# Convert vehicle year to int
df.vehicle_year = df.vehicle_year.astype(int)

In [41]:
# Save final df to a new CSV
df.to_csv('cleaned_and_filtered_data_full.csv')

In [42]:
df.head()

,plate_id,registration_state,violation_code,vehicle_body_type,vehicle_make,borough,vehicle_color,vehicle_year
timestamp,,,,,,,,
2016-01-01 00:40:00,GXA7174,NY,FIRE HYDRANT,SUBN,INFIN,Bronx,BLUE,2009
2016-01-01 00:50:00,GYK1668,NY,NO PARKING ZONE,SUBN,FORD,Bronx,GRAY,2003
2016-01-01 01:34:00,HDU1328,NY,FIRE HYDRANT,SUBN,NISSA,Manhattan,BLACK,2011
2016-01-01 01:53:00,GPU1150,NY,FIRE HYDRANT,SUBN,NISSA,Brooklyn,BLACK,1999
2016-01-01 02:41:00,FYK7567,NY,FIRE HYDRANT,SUBN,BMW,Bronx,WHITE,2014
